In [8]:

import numpy as np
import pandas as pd

class VolatilityCalculator:
    """
    Calculate historical volatility from a price series.
    """
    def __init__(self, prices, trading_days_per_year=252):
        # Accept list, numpy array, or pandas Series
        self.prices = pd.Series(prices).astype(float)
        self.trading_days = trading_days_per_year

    def log_returns(self, k=1):
        """
        Compute k-period log returns: ln(P_t / P_{t-k})
        """
        return np.log(self.prices / self.prices.shift(k)).dropna()

    def volatility(self, k=1):
        """
        Compute annualized volatility based on k-period log returns.
        Annualization factor = sqrt(trading_days_per_year / k).
        """
        returns = self.log_returns(k)
        scaling = np.sqrt(self.trading_days / k)
        return returns.std(ddof=1) * scaling


In [9]:
np.random.seed(42)
dates = pd.date_range(start="2024-01-01", periods=252, freq="B")
mu, sigma = 0.0005, 0.02
daily_returns = np.random.normal(loc=mu, scale=sigma, size=len(dates))
prices = 100 * np.exp(np.cumsum(daily_returns))

df = pd.DataFrame({
    "Price": prices,
    "LogRet_1d": np.log(prices / np.roll(prices, 1)),
    "LogRet_5d": np.log(prices / np.roll(prices, 5))
}, index=dates).dropna()

In [11]:


vc = VolatilityCalculator(df["Price"])
daily_vol = vc.volatility(k=1)
weekly_vol = vc.volatility(k=5)
monthly_vol = vc.volatility(k=21)

print(f"Annualized daily volatility (k=1): {daily_vol:.2%}")
print(f"Annualized weekly volatility (k=5): {weekly_vol:.2%}")
print(f"Annualized monthly volatility (k=21): {monthly_vol:.2%}")

Annualized daily volatility (k=1): 30.75%
Annualized weekly volatility (k=5): 28.23%
Annualized monthly volatility (k=21): 29.05%
